In [ ]:
from pyspark.sql import SparkSession

scala_version = '2.12'  
spark_version = '3.5.3'
packages = [
    f'org.apache.spark:spark-sql-kafka-0-10_{scala_version}:{spark_version}',
    'org.apache.kafka:kafka-clients:3.9.0'
]
spark = SparkSession.builder\
   .master("local")\
   .appName("kafka-example")\
   .config("spark.jars.packages", ",".join(packages))\
   .getOrCreate()
spark

In [ ]:
# TODO: Replace with real list of data
data = ['Hello', 'World']
df = spark.createDataFrame([{'value': v} for v in data])

From the host, create the topic

```sh
/opt/kafka/bin/kafka-topics.sh --create --bootstrap-server localhost:9092 --topic foobar --partitions=1 --replication-factor=1
```

In [ ]:
topic_name = 'foobar'
df.write.format("kafka")\
  .option("kafka.bootstrap.servers", "localhost:9092")\
  .option("topic", topic_name)\
  .save()

In [ ]:
from pyspark.sql.functions import col, concat, lit

kafkaDf = spark.read.format("kafka")\
  .option("kafka.bootstrap.servers", "localhost:9092")\
  .option("subscribe", topic_name)\
  .option("startingOffsets", "earliest")\
  .load()
kafkaDf.select(
    concat(col("topic"), lit(':'), col("partition").cast("string")).alias("topic_partition"),
    col("offset"),
    col("value").cast("string")
).show()